In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
from sklearn.metrics import matthews_corrcoef
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge, LassoCV, RidgeCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor

In [2]:
trd = pd.read_csv('train.csv').drop('Index', 1)
tsd = pd.read_csv('test.csv').drop('Index', 1)

In [3]:
trd.head()

,Gender,Senior,Married,Dependents,Tenure,Phone,MultiplePhones,Internet,Security,Backup,Insurance,Support,TV,Movies,Contract,EBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,Yes,67,No,No phone service,DSL,Yes,Yes,Yes,Yes,Yes,No,Two year,No,Credit card (automatic),54.20,3623.95,False
1,Female,0,No,No,34,Yes,Yes,DSL,Yes,Yes,No,Yes,No,No,One year,Yes,Mailed check,64.40,2088.75,True
2,Female,1,Yes,No,72,Yes,Yes,Fiber optic,Yes,Yes,Yes,Yes,No,No,Two year,Yes,Bank transfer (automatic),96.60,6827.5,False
3,Female,0,Yes,Yes,63,Yes,No,DSL,Yes,Yes,Yes,Yes,No,No,Two year,No,Mailed check,63.55,4014.2,False
4,Male,0,Yes,Yes,69,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,No,One year,No,Credit card (automatic),95.75,6511.25,False


In [4]:
trdwd = pd.concat([
    trd.drop(['Gender', 'Internet', 'Contract'], axis=1),

    pd.get_dummies(trd['Internet'], prefix='Internet'),    
    pd.get_dummies(trd['Contract'], prefix='Contract')
], axis = 1)

trdwd = trdwd.replace(['Electronic check', 'Mailed check'], 0)
trdwd = trdwd.replace(['Bank transfer (automatic)', 'Credit card (automatic)'], 1)

trdwd = trdwd.replace(['No phone service', 'No internet service'], 0)

trdwd = trdwd.replace(['Yes', True, 'True'], 1)
trdwd = trdwd.replace(['No', False, 'False'], 0)
trdwd = trdwd.replace(['', ' '], 0)

trdwd

,Senior,Married,Dependents,Tenure,Phone,MultiplePhones,Security,Backup,Insurance,Support,...,PaymentMethod,MonthlyCharges,TotalCharges,Churn,Internet_DSL,Internet_Fiber optic,Internet_No,Contract_Month-to-month,Contract_One year,Contract_Two year
0,0,1,1,67,0,0,1,1,1,1,...,1,54.20,3623.95,False,1,0,0,0,0,1
1,0,0,0,34,1,1,1,1,0,1,...,0,64.40,2088.75,True,1,0,0,0,1,0
2,1,1,0,72,1,1,1,1,1,1,...,1,96.60,6827.5,False,0,1,0,0,0,1
3,0,1,1,63,1,0,1,1,1,1,...,0,63.55,4014.2,False,1,0,0,0,0,1
4,0,1,1,69,1,1,0,1,1,0,...,1,95.75,6511.25,False,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3586,0,0,0,5,1,0,0,0,0,0,...,0,80.20,384.25,False,0,1,0,1,0,0
3587,0,1,1,71,1,0,1,0,1,0,...,1,77.55,5574.35,False,1,0,0,0,0,1
3588,0,1,0,68,1,1,0,1,1,0,...,1,86.45,5762.95,False,0,1,0,0,1,0
3589,0,1,1,12,1,0,0,0,1,0,...,1,58.35,740.55,False,1,0,0,1,0,0


In [5]:
X, Y = trdwd.drop(['Churn'], axis=1), trdwd['Churn']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=10)

# model = RandomForestRegressor(random_state = 10, n_estimators = 100)
# model.fit(X_train, Y_train)
# predictions = model.predict(X_test)
# predictions

# lasso = Lasso()
# lasso.fit(X_train, Y_train)
# predictions = lasso.predict(X_test)
# predictions

ridge = Ridge()
ridge.fit(X_train, Y_train)
predictions = ridge.predict(X_test)
predictions

array([ 0.3133548 ,  0.4121191 ,  0.0262982 , ...,  0.12974591,
       -0.04490316,  0.18967797])

In [6]:
act = list(map(lambda x: True if x > 0.5 else False, predictions))

In [7]:
exp = list(Y_test)
matthews_corrcoef(act, exp)

0.43183202214261557

In [8]:
# tt = 0
# tf = 0
# ff = 0
# ft = 0
# for i in range(len(act)):
#     if bool(act[i]) and bool(exp[i]):
#         tt += 1
#     if bool(act[i]) and not bool(exp[i]):
#         tf += 1
#     if not bool(act[i]) and not bool(exp[i]):
#         ff += 1
#     if not bool(act[i]) and bool(exp[i]):
#         ft += 1

# print('tt: ', tt/len(act)) 
# print('ff: ', ff/len(act))
# print('tf: ', tf/len(act))
# print('ft: ', ft/len(act))
# print('!: ', (tt + ff) / len(act))

In [9]:
tsd.head()

,Gender,Senior,Married,Dependents,Tenure,Phone,MultiplePhones,Internet,Security,Backup,Insurance,Support,TV,Movies,Contract,EBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,Male,0,No,No,58,Yes,No,Fiber optic,Yes,Yes,No,No,Yes,Yes,One year,No,Bank transfer (automatic),99.15,5720.95
1,Female,0,Yes,Yes,26,Yes,No,Fiber optic,No,No,No,No,Yes,Yes,Month-to-month,Yes,Electronic check,90.80,2361.8
2,Female,0,No,No,26,Yes,No,Fiber optic,Yes,No,Yes,Yes,No,No,One year,Yes,Credit card (automatic),87.15,2274.1
3,Male,0,No,No,12,Yes,Yes,DSL,Yes,Yes,No,Yes,No,No,Month-to-month,Yes,Electronic check,67.25,832.3
4,Female,0,No,No,1,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Mailed check,75.05,75.05


In [10]:
tsdwd = pd.concat([
    tsd.drop(['Gender', 'Internet', 'Contract'], axis=1),

    pd.get_dummies(tsd['Internet'], prefix='Internet'),    
    pd.get_dummies(tsd['Contract'], prefix='Contract'),
], axis = 1)

tsdwd = tsdwd.replace(['Electronic check', 'Mailed check'], 0)
tsdwd = tsdwd.replace(['Bank transfer (automatic)', 'Credit card (automatic)'], 1)

tsdwd = tsdwd.replace(['No phone service', 'No internet service'], 0)

tsdwd = tsdwd.replace(['Yes', True, 'True'], 1)
tsdwd = tsdwd.replace(['No', False, 'False'], 0)
tsdwd = tsdwd.replace(['', ' '], 0)

tsdwd

,Senior,Married,Dependents,Tenure,Phone,MultiplePhones,Security,Backup,Insurance,Support,...,EBilling,PaymentMethod,MonthlyCharges,TotalCharges,Internet_DSL,Internet_Fiber optic,Internet_No,Contract_Month-to-month,Contract_One year,Contract_Two year
0,0,0,0,58,1,0,1,1,0,0,...,0,1,99.15,5720.95,0,1,0,0,1,0
1,0,1,1,26,1,0,0,0,0,0,...,1,0,90.80,2361.8,0,1,0,1,0,0
2,0,0,0,26,1,0,1,0,1,1,...,1,1,87.15,2274.1,0,1,0,0,1,0
3,0,0,0,12,1,1,1,1,0,1,...,1,0,67.25,832.3,1,0,0,1,0,0
4,0,0,0,1,1,1,0,0,0,0,...,1,0,75.05,75.05,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2390,0,0,0,62,0,0,1,1,1,1,...,1,1,43.70,2618.3,1,0,0,0,0,1
2391,0,1,0,2,1,0,0,0,0,0,...,0,1,20.10,43.15,0,0,1,1,0,0
2392,0,1,1,23,1,1,0,0,0,0,...,1,0,83.75,1849.95,0,1,0,1,0,0
2393,1,1,0,53,1,1,0,1,0,0,...,1,0,101.90,5549.4,0,1,0,1,0,0


In [11]:
predictions_finally = ridge.predict(tsdwd)
predictions_finally

array([0.05317277, 0.55048742, 0.20976061, ..., 0.57086526, 0.47008673,
       0.35630397])

In [12]:
act = list(map(lambda x: True if x > 0.46 else False, predictions_finally))
df = pd.DataFrame(act)
df

,0
0,False
1,True
2,False
3,False
4,True
...,...
2390,False
2391,False
2392,True
2393,True


In [13]:
df.to_csv('out.csv', index=True)